In [1]:
%matplotlib inline  
import mxnet as mx
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt

from vision.cnn import resnet

In [2]:
data_dir = '/home/ycli/Downloads/cifar-10/cifar-10-batches-py'

train_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']
test_file = 'test_batch'

In [3]:
data = []
label = []

for train_file in train_files:
    with open(os.path.join(data_dir, train_file), 'rb') as fo:
        tmp = pickle.load(fo, encoding='bytes')
        data.append(np.reshape(tmp[b'data'], (tmp[b'data'].shape[0], 3, 32, 32)))
        label.extend(tmp[b'labels'])

In [4]:
data = np.concatenate(data, axis=0)

In [5]:
data.shape

(50000, 3, 32, 32)

In [6]:
test_data = []
test_label = []


with open(os.path.join(data_dir, test_file), 'rb') as fo:
        tmp = pickle.load(fo, encoding='bytes')
        test_data = np.reshape(tmp[b'data'], (tmp[b'data'].shape[0], 3, 32, 32))
        test_label = tmp[b'labels']

In [7]:
oh_label = mx.nd.one_hot(mx.nd.array(label), 10)
train_iter = mx.io.NDArrayIter(data, oh_label, batch_size=5, shuffle=True)
oh_label = mx.nd.one_hot(mx.nd.array(test_label), 10)
val_iter = mx.io.NDArrayIter(test_data,oh_label, batch_size=5)

In [8]:
r18 = resnet.ResnetV1(resnet.residual_block_v1, [2, 2, 2, 2], [64, 64, 128, 256, 512], classes=10)

r18_sym = mx.sym.SoftmaxOutput(data=r18.output)

In [9]:
r18_model = mx.mod.Module(symbol=r18_sym, context=mx.gpu(0))

/home/ycli/opt/miniconda3/envs/dl/lib/python3.5/site-packages/mxnet/module/base_module.py:55: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
	softmaxoutput0_label
  warnings.warn(msg)


In [10]:
r18_model.fit(train_data=train_iter, eval_data=val_iter, optimizer='sgd', 
              optimizer_params={'learning_rate': 0.1}, eval_metric='acc', 
              batch_end_callback = mx.callback.Speedometer(5, 100),
              epoch_end_callback = mx.callback.log_train_metric(period=200), num_epoch=3)

RuntimeError: simple_bind error. Arguments:
softmax_label: (5, 10)
data: (5, 3, 32, 32)
[23:52:05] src/storage/storage.cc:119: Check failed: e == cudaSuccess || e == cudaErrorCudartUnloading CUDA: unknown error

Stack trace returned 10 entries:
[bt] (0) /home/ycli/opt/miniconda3/envs/dl/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x39008a) [0x7ff745cf308a]
[bt] (1) /home/ycli/opt/miniconda3/envs/dl/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x3906c1) [0x7ff745cf36c1]
[bt] (2) /home/ycli/opt/miniconda3/envs/dl/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x31aeebb) [0x7ff748b11ebb]
[bt] (3) /home/ycli/opt/miniconda3/envs/dl/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x31b2522) [0x7ff748b15522]
[bt] (4) /home/ycli/opt/miniconda3/envs/dl/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x31b3e4f) [0x7ff748b16e4f]
[bt] (5) /home/ycli/opt/miniconda3/envs/dl/lib/python3.5/site-packages/mxnet/libmxnet.so(+0x2b1ee53) [0x7ff748481e53]
[bt] (6) /home/ycli/opt/miniconda3/envs/dl/lib/py